In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.ensemble import VotingClassifier

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

1. Read Dataset 
===

In [3]:
xtest = np.array(pd.read_csv("../dataset/xtest.txt"))
xtrain = np.array(pd.read_csv("../dataset/xtrain.txt"))
ytest_original = np.array(pd.read_csv("../dataset/ytest.txt")).ravel()
ytrain_original = np.array(pd.read_csv("../dataset/ytrain.txt")).ravel()

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()


real_xtest=xtest
real_ytest=ytest_original

x=xtrain
y=ytrain_original


<IPython.core.display.Javascript object>

In [4]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval

<IPython.core.display.Javascript object>

In [5]:

original_scores=[]
trial1_scores=[]
trial2_scores=[]
trial3_scores=[]
trial4_scores=[]
trial5_scores=[]
trial6_scores=[]

from sklearn.model_selection import KFold 
kf = KFold(n_splits=10, random_state=randomseed, shuffle=True)
kf.get_n_splits(x)

print(kf)

data_cv=[]

for train_index, test_index in kf.split(x):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]    
    data_cv.append([[X_train, X_test],[y_train, y_test]])


for i in range(len(data_cv)):
    print('************************** ==> ',i)
    xtrain=data_cv[i][0][0]
    ytrain_original =data_cv[i][1][0] 
    xtest= real_xtest#data_cv[i][0][1]
    ytest_original= real_ytest#data_cv[i][1][1]
    
    
    # xtrain,xtest,ytrain_original,ytest_original=train_test_split(x,y,random_state=randomseed,test_size=0.3) 
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy() 
    
    # ytrain, ytest = swapcolumns(ytrain, ytest, 2)
    
    
    
    clf=[]
    acc=[]
    finalacc=[]
    ypredproba_all=[]
    ypredconfprob_all=[]
    
    rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
    rf.fit(xtrain,ytrain)
    print('original score',m.f1_score(ytest,rf.predict(xtest),average='weighted'))
    original_scores.append(m.f1_score(ytest,rf.predict(xtest),average='weighted'))
    
    #================================================= 
    
    # Class 0
    # ===========================
    ytrain = ytrain_original.copy()
    ytest = ytest_original.copy()
    ytrain,ytest= swapcolumns(ytrain,ytest,0)
    #=================================================
    
    
    xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
    xgbc.fit(xtrain,ytrain)

    xgbpred=xgbc.predict(xtest)
    print(m.f1_score(ytest,xgbpred,average='weighted'))


    clf.append(xgbc)
    acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
    ypredproba_all.append(xgbc.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,xgbpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)
    
    
    # =================================================
    # classs 1
    # =================================================
    ytrain = ytrain_original.copy()
    ytest = ytest_original.copy()
    ytrain, ytest = swapcolumns(ytrain, ytest, 1)
    # =================================================
    
    xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
    xgbc.fit(xtrain,ytrain)

    xgbpred=xgbc.predict(xtest)
    print(m.f1_score(ytest,xgbpred,average='weighted'))


    clf.append(xgbc)
    acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
    ypredproba_all.append(xgbc.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,xgbpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)
            
    #=================================================
    # classs 2
    #=================================================
    
    ytrain=ytrain_original.copy()
    ytest=ytest_original.copy()
    ytrain,ytest= swapcolumns(ytrain,ytest,2)
    
    #=================================================
    xgbc=xgb.XGBClassifier(random_state=randomseed,n_estimators=100)
    xgbc.fit(xtrain,ytrain)

    xgbpred=xgbc.predict(xtest)
    print(m.f1_score(ytest,xgbpred,average='weighted'))


    clf.append(xgbc)
    acc.append(m.f1_score(ytest,xgbpred,average='weighted'))
    ypredproba_all.append(xgbc.predict_proba(xtest))

    confmat=m.confusion_matrix(ytest,xgbpred)
    confsumh=np.sum(confmat,axis=1)
    propconfmat=confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i]= 100*propconfmat[i]/confsumh[i] 
    ypredconfprob_all.append(propconfmat/100)
    
    # #=================================================
    
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues(acc)
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    # accuracy_score 0.6433333333333333
    
    trial1_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    # accuracy_score 0.6783333333333333
    # accuracy_score 0.715
    # accuracy_score 0.7216666666666667
    trial2_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0] * weightvalga[0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1]
    finalcol[:, 1] = ypredproba_all[1][:, 0] * weightvalga[1] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1]
    finalcol[:, 2] = ypredproba_all[2][:, 0] * weightvalga[2] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial3_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1] ,
        -ypredconfprob_all[0][0][1],
        
        ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
        -ypredconfprob_all[1][0][1],
        
        ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1],
        -ypredconfprob_all[2][0][1],
        
        ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0] + ypredproba_all[0][:, 1]*weightvalga[1]
    finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[2] + ypredproba_all[1][:, 1]*weightvalga[3]
    finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[4] + ypredproba_all[2][:, 1]*weightvalga[5]
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    
    trial4_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================

    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] ,
        ypredconfprob_all[1][0][0] ,
        ypredconfprob_all[2][0][0] ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] = weightvalga[0] * (ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
    finalcol[:, 1] = weightvalga[1] * (ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
    finalcol[:, 2] = weightvalga[2] * (ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial5_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================
    
    import calculateWeightUsingGa2 as aresult
    weightvalga=aresult.getbestvalues([
        ypredconfprob_all[0][0][0] ,
        ypredconfprob_all[1][0][0] ,
        ypredconfprob_all[2][0][0] ])
    
    finalcol = np.zeros((ytest.shape[0], 3))
    finalcol[:, 0] =  (weightvalga[0] *ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
    finalcol[:, 1] = (weightvalga[1] * ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
    finalcol[:, 2] =  (weightvalga[2] *ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])
    
    finalpred = np.argmax(finalcol, axis=1)
    
    ytest = ytest_original.copy()
    print('f1_score',m.f1_score(ytest, finalpred,average='weighted'))
    trial6_scores.append(m.f1_score(ytest, finalpred,average='weighted'))
    
    # #=================================================

    


KFold(n_splits=10, random_state=42, shuffle=True)
************************** ==>  0
original score 0.7102197704194443
0.8175381263616558
0.7356439061960534
0.9149465423859273
0
1
2
3
4
5
6
7
8
9
2.4054217761139216 [0.96478946 0.95547893 0.99955549]
f1_score 0.731260328000966
0
1
2
3
4
5
6
7
8
9
1.5210659407112133 [0.92575719 0.8831117  0.99865188]
f1_score 0.7230225974554912
0
1
2
3
4
5
6
7
8
9
1.5564571253174018 [0.97571779 0.90729305 0.99738795]
f1_score 0.7442248227962514
0
1
2
3
4
5
6
7
8
9
0.6923915078800983 [0.94801547 0.77738735 0.86499056 0.75938175 0.97791155 0.848693  ]
f1_score 0.47817166305710707
0
1
2
3
4
5
6
7
8
9
2.1084753212517664 [0.9705244  0.94173244 0.98666024]
f1_score 0.733024566178768
0
1
2
3
4
5
6
7
8
9
2.12303957883268 [0.96454716 0.95576391 0.99538207]
f1_score 0.7409589395085578
************************** ==>  1
original score 0.7043733398384562
0.8159829870888571
0.7322763126444108
0.911381922058862
0
1
2
3
4
5
6
7
8
9
2.3714094780208717 [0.98823266 0.903162

<IPython.core.display.Javascript object>

In [6]:
print("original_scores", np.mean(original_scores), np.std(original_scores))
print("trial1_scores", np.mean(trial1_scores), np.std(trial1_scores))
print("trial2_scores", np.mean(trial2_scores), np.std(trial2_scores))
print("trial3_scores", np.mean(trial3_scores), np.std(trial3_scores))
print("trial2_scores", np.mean(trial4_scores), np.std(trial4_scores))
print("trial3_scores", np.mean(trial5_scores), np.std(trial5_scores))

original_scores 0.7095634797794663 0.005925041522788456
trial1_scores 0.7291536335658405 0.004144340596880795
trial2_scores 0.7282527130633112 0.004531858072106312
trial3_scores 0.7400670922401347 0.0034557980778551134
trial2_scores 0.47250287384940953 0.05654394923913593
trial3_scores 0.7283631391273379 0.003538339157263827


<IPython.core.display.Javascript object>